## Problem 48 - Self Powers

<p>The series, $1^1 + 2^2 + 3^3 + \cdots + 10^{10} = 10405071317$.</p>
<p>Find the last ten digits of the series, $1^1 + 2^2 + 3^3 + \cdots + 1000^{1000}$.</p>


In [3]:
%%time

s = 0
for x in range(1, 1001, 1):
    s+= x**x

print(f"The answer is : {str(s)[-10:]}")

The answer is : 9110846700
CPU times: user 6.68 ms, sys: 0 ns, total: 6.68 ms
Wall time: 6.62 ms


## Problem 49 - Prime Permutations

<p>The arithmetic sequence, $1487, 4817, 8147$, in which each of the terms increases by $3330$, is unusual in two ways: (i) each of the three terms are prime, and, (ii) each of the $4$-digit numbers are permutations of one another.</p>
<p>There are no arithmetic sequences made up of three $1$-, $2$-, or $3$-digit primes, exhibiting this property, but there is one other $4$-digit increasing sequence.</p>
<p>What $12$-digit number do you form by concatenating the three terms in this sequence?</p>


In [45]:
from sympy import isprime
import numpy as np
from scipy.spatial.distance import cdist

#find four-digit-primes
fdps = np.array([x for x in range(1000, 10000,1) if isprime(x)])

In [84]:
def check_gap(fdp: int, gap: int):
    """Checks if for a four digit prime and a given gap, 
    the numbers found by adding or subtracting the gap are prime
    and also comprised of the same set of digits"""
    if set(str(fdp)) != set(str(fdp+gap)):
        return False
    
    if set(str(fdp)) != set(str(fdp-gap)):
        return False
    if not isprime(fdp+gap) or not isprime(fdp-gap):
        return False
    return True
        

In [82]:
#for every four digit prime
for fdp in fdps:
    # find all symmetrical gaps by looki
    diffs, counts = np.unique(np.abs(fdps - fdp), return_counts=True)
    idxs = np.argwhere(counts == 2).flatten()
    gaps_to_check = diffs[idxs]
    for gap in gaps_to_check:
        if check_gap(fdp, gap):
            print(f"{fdp-gap}, {fdp}, {fdp+gap} with a gap of {gap}")

1487, 4817, 8147 with a gap of 3330
2969, 6299, 9629 with a gap of 3330


In [62]:
4817+114

4931

In [63]:
set(str(1234))

{'1', '2', '3', '4'}